###  Testing with SDK v2

In [1]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.identity import DefaultAzureCredential
import sys

# Enter details of your AML workspace
subscription_id = "840b5c5c-3f4a-459a-94fc-6bad2a969f9d"
resource_group = "ml"
workspace = "ws02ent"
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

sys.path.append("../") # go to parent dir

In [2]:
from src.ray_on_aml.core import Ray_On_AML

import logging
ray_on_aml =Ray_On_AML(ml_client=ml_client, compute_cluster ="ds11")
#Note that if you need to customize the pip installation of the cluster, you also needs to support the ray package e.g. ray[data] which 
#match the version of the ray package(s) in your compute instance. If you don't specify pip_packages then ray[default] is inserted 
#automatically

# For use as client mode, uncomment these lines
ray = ray_on_aml.getRay(ci_is_head=True, num_node=2,pip_packages=["ray[data]","fastparquet", "azureml-mlflow", "pyarrow==6.0.1", "dask==2022.2.0", "adlfs", "fsspec"])
# client = ray.init(f"ray://{ray_on_aml.headnode_private_ip}:10001")

#use CI as head node

# ray = ray_on_aml.getRay(ci_is_head=True, num_node=2)

Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/bin//ray", line 8, in <module>
    sys.exit(main())
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/scripts/scripts.py", line 2588, in main
    return cli()
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1130, in __call__
    return self.main(*args, **kwargs)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1055, in main
    rv = self.invoke(ctx)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1657, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 1404, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/click/core.py", line 760, in invoke
    return __callback(*args, **kwargs)
  File "/anaconda/envs/azureml_py3

Waiting for cluster to start
...

(raylet, ip=10.0.0.6) /azureml-envs/azureml_118a0a3f1c505e56ae5dec61b113db5b/lib/python3.10/site-packages/ray/dashboard/agent.py:50: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.0.6)   aiogrpc.init_grpc_aio()
(raylet, ip=10.0.0.6) /azureml-envs/azureml_118a0a3f1c505e56ae5dec61b113db5b/lib/python3.10/site-packages/ray/dashboard/agent.py:468: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.0.6)   loop = asyncio.get_event_loop()
(raylet, ip=10.0.0.10) /azureml-envs/azureml_118a0a3f1c505e56ae5dec61b113db5b/lib/python3.10/site-packages/ray/dashboard/agent.py:50: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.0.10)   aiogrpc.init_grpc_aio()
(raylet, ip=10.0.0.10) /azureml-envs/azureml_118a0a3f1c505e56ae5dec61b113db5b/lib/python3.10/site-packages/ray/dashboard/agent.py:468: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.0.10)   loop = asyncio.get_event_loop()



 Cluster started successfully


AttributeError: 'Ray_On_AML' object has no attribute 'headnode_private_ip'

In [10]:
#Example of using input and output for interactive job
from azure.ai.ml import command, Input, Output
from src.ray_on_aml.core import Ray_On_AML
import logging
ray_on_aml =Ray_On_AML(ml_client=ml_client, compute_cluster ="ds11", verbosity=logging.INFO )

inputs={

    "ISDWeather": Input(
        type="uri_folder",
        path="azureml://datastores/adlsstore0001/paths/ISDWeather/year=2008",
    )
}

outputs={
    "output1": Output(
        type="uri_folder",
        path="azureml://datastores/adlsstore0001/paths/dev",
    ),
    "output2": Output(
        type="uri_folder",
        path="azureml://datastores/adlsstore0001/paths/dev",
    )
}

ray = ray_on_aml.getRay(inputs = inputs,outputs=outputs, num_node=2,pip_packages=["ray[data]", "fastparquet", "azureml-mlflow", "pyarrow==6.0.1", "dask==2022.2.0", "adlfs", "fsspec"])

Waiting cluster to start and return head node's ip
...

2022-12-14 19:54:36,690	WARNING worker.py:1839 -- The node with node id: d44347aed336b31268cbba5b0ee97a4a65d77d417baf581b77133e7c and address: 10.0.0.8 and node name: 10.0.0.8 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, preempted node, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.


.

2022-12-14 19:54:40,690	WARNING worker.py:1839 -- The node with node id: ed0005dbb2d47b243d5f1118cc8e9beffbc364746d336f3830aa2a28 and address: 10.0.0.9 and node name: 10.0.0.9 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, preempted node, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.


.........
 cluster is ready, head node ip  10.0.0.8


In [3]:

ray.cluster_resources()

{'object_store_memory': 10938155827.0,
 'node:10.0.0.4': 1.0,
 'CPU': 6.0,
 'memory': 24778350183.0,
 'node:10.0.0.6': 1.0,
 'node:10.0.0.10': 1.0}

In [4]:
ray_on_aml.shutdown()

Log channel is reconnecting. Logs produced while the connection was down can be found on the head node of the cluster in `ray_client_server_[port].out`
2022-12-14 22:17:01,329	WARNING dataclient.py:396 -- Encountered connection issues in the data channel. Attempting to reconnect.
2022-12-14 22:17:31,533	WARNING dataclient.py:403 -- Failed to reconnect the data channel


In [4]:
# from adlfs import AzureBlobFileSystem
# import dask.dataframe as dd
# from ray.util.dask import ray_dask_get, enable_dask_on_ray, disable_dask_on_ray
# enable_dask_on_ray()
# abfs = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")

# storage_options = {'account_name': 'azureopendatastorage'}
# ddf = dd.read_parquet('az://nyctlc/green/puYear=2019/puMonth=*/*.parquet', storage_options=storage_options)

data = ray.data.read_parquet(ray_on_aml.mount_points['ISDWeather'])

# data = ray.data.read_parquet("az://isdweatherdatacontainer/ISDWeather/year=2009", filesystem=abfs)
print(data.count())

(_get_read_tasks pid=274) [dataset]: Run `pip install tqdm` to enable progress reporting.


(raylet, ip=10.0.0.9) /azureml-envs/azureml_118a0a3f1c505e56ae5dec61b113db5b/lib/python3.10/site-packages/ray/dashboard/agent.py:50: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.0.9)   aiogrpc.init_grpc_aio()
(raylet, ip=10.0.0.9) /azureml-envs/azureml_118a0a3f1c505e56ae5dec61b113db5b/lib/python3.10/site-packages/ray/dashboard/agent.py:468: DeprecationWarning: There is no current event loop
(raylet, ip=10.0.0.9)   loop = asyncio.get_event_loop()


97581959


In [12]:
client.disconnect()

###  Testing with SDK v1

In [1]:
from azureml.core import Workspace
import sys
sys.path.append("../") # go to parent dir
import importlib


In [2]:
from src.ray_on_aml.core import Ray_On_AML
import ray
ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="ds11",maxnode=2)
# ray_on_aml.getRay(num_node=2,pip_packages=["torch==1.13.0", "azureml-mlflow"], ci_is_head=True)
ray = ray_on_aml.getRay(num_node=2,pip_packages=["ray[data]==2.1.0","fastparquet==2022.12.0", "azureml-mlflow==1.48.0", "pyarrow==6.0.1", "dask==2022.2.0", "adlfs==2022.11.2", "fsspec==2022.11.0"])

client = ray.init(f"ray://{ray_on_aml.headnode_private_ip}:10001")


Waiting cluster to start and return head node ip
..

In [4]:
ray.cluster_resources()

{'object_store_memory': 10911909886.0,
 'node:10.0.0.4': 1.0,
 'CPU': 6.0,
 'memory': 24723150441.0,
 'node:10.0.0.6': 1.0,
 'node:10.0.0.10': 1.0}

In [3]:

# Load data.
dataset = ray.data.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv").repartition(4)

train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

# Create a test dataset by dropping the target column.
test_dataset = valid_dataset.drop_columns(cols=["target"])
# Create a preprocessor to scale some columns.

import numpy as np

from ray.data.preprocessors import Concatenator, Chain, StandardScaler

# Create a preprocessor to scale some columns and concatenate the result.
# preprocessor = Chain(
#     StandardScaler(columns=["mean radius", "mean texture"]),
#     Concatenator(exclude=["target"], dtype=np.float32),
# )


RayTaskError(ImportError): [36mray::_get_read_tasks()[39m (pid=212508, ip=10.0.0.4)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/ray/data/read_api.py", line 1380, in _get_read_tasks
    reader = ds.create_reader(**kwargs)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/ray/data/datasource/file_based_datasource.py", line 219, in create_reader
    return _FileBasedDatasourceReader(self, **kwargs)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/ray/data/datasource/file_based_datasource.py", line 378, in __init__
    paths, self._filesystem = _resolve_paths_and_filesystem(paths, filesystem)
  File "/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/ray/data/datasource/file_based_datasource.py", line 666, in _resolve_paths_and_filesystem
    raise ImportError(
ImportError: Please install fsspec to read files from HTTP.

In [9]:
ray_on_aml.shutdown(end_all_runs=True)

{'node:10.0.0.4': 1.0,
 'memory': 4464080487.0,
 'CPU': 2.0,
 'object_store_memory': 2232040243.0}

In [ ]:
import torch
import torch.nn as nn
from torch.nn.modules.utils import consume_prefix_in_state_dict_if_present

from ray import train
from ray.air import session
from ray.air.config import ScalingConfig
from ray.train.torch import TorchCheckpoint, TorchTrainer


def create_model(input_features):
    return nn.Sequential(
        nn.Linear(in_features=input_features, out_features=16),
        nn.ReLU(),
        nn.Linear(16, 16),
        nn.ReLU(),
        nn.Linear(16, 1),
        nn.Sigmoid(),
    )


def train_loop_per_worker(config):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["num_epochs"]
    num_features = config["num_features"]

    # Get the Ray Dataset shard for this data parallel worker,
    # and convert it to a PyTorch Dataset.
    train_data = train.get_dataset_shard("train")
    # Create model.
    model = create_model(num_features)
    model = train.torch.prepare_model(model)

    loss_fn = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for cur_epoch in range(epochs):
        for batch in train_data.iter_torch_batches(
            batch_size=batch_size, dtypes=torch.float32
        ):
            # "concat_out" is the output column of the Concatenator.
            inputs, labels = batch["concat_out"], batch["target"]
            optimizer.zero_grad()
            predictions = model(inputs)
            train_loss = loss_fn(predictions, labels.unsqueeze(1))
            train_loss.backward()
            optimizer.step()
        loss = train_loss.item()
        session.report({"loss": loss}, checkpoint=TorchCheckpoint.from_model(model))


num_features = len(train_dataset.schema().names) - 1

trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={
        "batch_size": 128,
        "num_epochs": 20,
        "num_features": num_features,
        "lr": 0.001,
    },
    scaling_config=ScalingConfig(
        num_workers=2,  # Number of workers to use for data parallelism.
        use_gpu=False,
        trainer_resources={"CPU": 1},  # so that the example works on Colab.
    ),
    datasets={"train": train_dataset},
    preprocessor=preprocessor,
)
# Execute training.
result = trainer.fit()
print(f"Last result: {result.metrics}")
# Last result: {'loss': 0.6559339960416158, ...}